# kaggle实战 狗的品种识别

In [1]:
import torch
from d2l import torch as d2l
import collections
import math
import os
import shutil  # shutil库用于文件和文件夹的高级操作，例如复制、移动、删除等
import pandas as pd
import torch
import torchvision
from torch import nn

In [2]:
from torch.nn.parallel import data_parallel


d2l.DATA_HUB['dog_tiny'] = (d2l.DATA_URL + 'kaggle_dog_tiny.zip', '0cb91d09b814ecdc07b50f31f8dcad3e81d6a86d')
print(d2l.DATA_HUB)

demo = True

if demo:
    data_dir = d2l.download_extract('dog_tiny')
else:
    data_dir = os.path.join('..', 'data', 'dog-breed-identification')

{'airfoil': ('http://d2l-data.s3-accelerate.amazonaws.com/airfoil_self_noise.dat', '76e5be1548fd8222e5074cf0faae75edff8cf93f'), 'hotdog': ('http://d2l-data.s3-accelerate.amazonaws.com/hotdog.zip', 'fba480ffa8aa7e0febbb511d181409f899b9baa5'), 'banana-detection': ('http://d2l-data.s3-accelerate.amazonaws.com/banana-detection.zip', '5de26c8fce5ccdea9f91267273464dc968d20d72'), 'voc2012': ('http://d2l-data.s3-accelerate.amazonaws.com/VOCtrainval_11-May-2012.tar', '4e443f8a2eca6b1dac8a6c57641b67dd40621a49'), 'cifar10_tiny': ('http://d2l-data.s3-accelerate.amazonaws.com/kaggle_cifar10_tiny.zip', '2068874e4b9a9f0fb07ebe0ad2b29754449ccacd'), 'dog_tiny': ('http://d2l-data.s3-accelerate.amazonaws.com/kaggle_dog_tiny.zip', '0cb91d09b814ecdc07b50f31f8dcad3e81d6a86d'), 'ptb': ('http://d2l-data.s3-accelerate.amazonaws.com/ptb.zip', '319d85e578af0cdc590547f26231e4e31cdf1e42'), 'glove.6b.50d': ('http://d2l-data.s3-accelerate.amazonaws.com/glove.6B.50d.zip', '0b8703943ccdb6eb788e6f091b8946e82231bc4d'), 

In [3]:
def reorg_dog_data(data_dir, valid_ratio):
    labels = d2l.read_csv_labels(os.path.join(data_dir, 'labels.csv'))
    d2l.reorg_train_valid(data_dir, labels, valid_ratio)
    d2l.reorg_test(data_dir)

batch_size = 32 if demo else 128
valid_ratio = 0.1
reorg_dog_data(data_dir, valid_ratio)

In [4]:
transform_train = torchvision.transforms.Compose([
    # 随机裁剪一个区域并缩放到224x224，scale控制裁剪区域的面积比例，ratio控制宽高比
    torchvision.transforms.RandomResizedCrop(224, scale=(0.08,1.0), ratio=(3.0/4.0, 4.0/3.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    # 这一行通过ColorJitter变换对图片的亮度、对比度、饱和度进行随机调整，从而增强训练数据的多样性，提高模型的泛化能力。
    torchvision.transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

transform_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

In [5]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_test) for folder in ['valid', 'test']]

In [6]:
train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [ ]:
def get_net(devices):
    """
    构建用于fine-tune的神经网络。

    该函数首先加载预训练的ResNet34模型，并将其特征层部分（features）作为新模型的主干。
    然后，在此基础上新建一个输出层(output_new)，用于狗品种分类。
    为了加速训练和防止过拟合，预训练主干的参数被冻结（不进行反向传播）。
    所有部件移动到所指定的设备上进行运算。

    参数:
        devices (list): 一组可用的计算设备，比如['cuda:0', 'cuda:1']。

    返回:
        nn.Sequential: 构造好的模型，包含固定参数的特征提取部分和可训练的自定义输出层。
    """
    # 新建一个空的容器，便于后续组合主干与输出层
    finetune_net = nn.Sequential()

    # 加载resnet34的预训练模型，作为特征提取器
    finetune_net.features = torchvision.models.resnet34(pretrained=True)
    # 打印参数名以及参数信息，帮助了解模型结构
    print(finetune_net.named_parameters())

    # 定义输出层先将1000维（ImageNet类别数）的输出映射到256维，
    # 添加ReLU激活，再映射为120维（狗品种数），实现迁移到新数据集
    finetune_net.output_new = nn.Sequential(
        nn.Linear(1000, 256),
        nn.ReLU(),
        nn.Linear(256, 120)
    )

    # 将模型加载到指定设备（如GPU），提升运算速度
    finetune_net = finetune_net.to(device=devices[0])

    # 冻结特征提取部分的参数，不参与训练
    for param in finetune_net.features.parameters():
        param.requires_grad = False
        
    # 返回组合好的网络
    return finetune_net

In [ ]:
loss = nn.CrossEntropyLoss()
def evaluate_loss(data_iter, net, devices):
    l_sum, n = 0.0, 0
    for features, labels in data_iter:
        features, labels = features.to(devices[0]), labels.to(devices[0])
        output = net(features)
        l = loss(output, labels)
        l_sum += l.sum()
        n += labels.numel()
    return (l_sum/n).to('cpu')

In [ ]:
def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay):
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    # 优化器只优化需要梯度的参数（即requires_grad=True的参数），设置学习率、动量和权重衰减
    trainer = torch.optim.SGD(
        (param for param in net.parameters() if param.requires_grad), 
        lr=lr, 
        momentum=0.9, 
        weight_decay=wd
    )
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss']
    if valid_iter is not None:
        legend.append('valid loss')
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=legend)
    for epoch in range(num_epochs):
        metric = d2l.Accumulator(2)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            features, labels = features.to(devices[0]), labels.to(devices[0])
            trainer.zero_grad()
            out_put = net(features)
            l = loss(out_put, labels).sum()
            l.backward()
            trainer.step()
            metric.add(l, labels.shape[0])
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[1], None))
        measures = f'train loss {metric[0] / metric[1]:.3f}'
        if valid_iter is not None:
            valid_loss = evaluate_loss(valid_iter, net, devices)
            animator.add(epoch + 1, (None, valid_loss.detach().cpu()))
        scheduler.step()
    if valid_iter is not None:
        measures += f', valid loss {valid_loss:.3f}'
    print(measures + f'\n{metric[1] * num_epochs / timer.sum():.1f}'
          f' examples/sec on {str(devices)}')

In [ ]:
devices, num_epochs, lr, wd = d2l.try_all_gpus(), 10, 1e-4, 1e-4
lr_period, lr_decay, net = 2, 0.9, get_net(devices)

train(net, train_iter=train_iter, valid_iter=valid_iter, num_epochs=num_epochs, lr=lr, wd=wd, devices=devices, lr_period=lr_period, lr_decay=lr_decay)

In [ ]:
net = get_net(devices)
train(net, train_valid_iter, None, num_epochs, lr, wd, devices, lr_period,
      lr_decay)

preds = []
for data, label in test_iter:
    output = torch.nn.functional.softmax(net(data.to(devices[0])), dim=1)
    preds.extend(output.cpu().detach().numpy())
ids = sorted(os.listdir(
    os.path.join(data_dir, 'train_valid_test', 'test', 'unknown')))
with open('submission.csv', 'w') as f:
    f.write('id,' + ','.join(train_valid_ds.classes) + '\n')
    for i, output in zip(ids, preds):
        f.write(i.split('.')[0] + ',' + ','.join(
            [str(num) for num in output]) + '\n')